In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import torch

from torch_sparse import SparseTensor, matmul

import matplotlib.pyplot as plt
from torch_geometric.data import HeteroData

C:\Users\Lucas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


# Data prep

In [2]:
df = pd.read_csv('./amazon/Beauty.csv').rename({"user_id":"u", "item_id":"i", "time":"t"}, axis=1)
df.head()
display(df)

,u,i,t
0,0,0,476496000
1,1,0,486432000
2,2,1,482803200
3,3,1,474422400
4,4,1,475372800
...,...,...,...
394903,15569,57287,496454400
394904,6783,57287,497232000
394905,35430,57288,496800000
394906,3542,57288,496886400


In [3]:
# Prep
def refine_time(data):
    """
    assures items bought by a user don't have the exact same time
    5, 1, 2, 2, 8 -> 1, 2, 3, 5, 8
    """
    
    data = data.sort_values(['t'], kind='mergesort')
    time_seq = data['t'].values
    time_gap = 1
    
    for i, da in enumerate(time_seq[0:-1]):
        if time_seq[i] == time_seq[i+1] or time_seq[i] > time_seq[i+1]:
            time_seq[i+1] = time_seq[i+1] + time_gap
            time_gap += 1
            
    data['t'] = time_seq
    
    return  data

def remove_less_than_n_transactions_users(dataf, n):
    transactions_per_customer = dataf['u'].value_counts()
    
    valid_customers = transactions_per_customer[transactions_per_customer>=n].index
    
    return dataf[dataf['u'].isin(valid_customers)]

print("Re-ordering and fixing time sequences...")
df = df.groupby('u').apply(refine_time).reset_index(drop=True)
df['t'] = df['t'].astype('int64')


# This does not work yet since the u's and i's need to be remapped to a continuous range again
# min_n = 5
# print(f"Removing users with less than {min_n} transactions")
# df = remove_less_than_n_transactions_users(df, min_n)


df

Re-ordering and fixing time sequences...


,u,i,t
0,0,12887,473731200
1,0,49582,475372800
2,0,0,476496000
3,0,4732,476496001
4,0,5760,476496002
...,...,...,...
394903,52201,57191,493689601
394904,52202,57190,493603200
394905,52202,57191,493603201
394906,52203,57277,490924800


# Sample algoritme

We sample efficiently by splitting the process up in 3 steps:

## Step 1 - Create 'dictionary'

Create dictionary of n recent interactions of each user/item

made as a big numpy array `u_connections` and `i_connections` met shape (u+1, n) en (i+1, n).

index 0 is om aan te geven dat ie geen connecties meer heeft

dus stel je wil de items van user [5, 6, 7, 8] weten dan kan je doen u_connections[6, 7, 8, 9]

u_connections = [[     0      0      0 ...      0      0      0]
 [    32     33     34 ...     39     40     41]
 [    42     43     44 ...      0      0      0]
 ...
 [394902 394903      0 ...      0      0      0]
 [394904 394905      0 ...      0      0      0]
 [394906 394907      0 ...      0      0      0]]
 
 
## Step 2 - get_user_network

Tweede stap is om het sample algoritme toe te passen, oftwel steeds de items van de users, dan de users van die items, enzovoort. Bij de vorige step hebben we stiekem ook opgeslagen welke transactie nummers van de aankopen er bij horen, en die slaan we op. Dus nu kunnen we de transacties verzamelen die gesampled zijn als edges. En dan eindigen dus we met een subset van de hele dataset, die beperkt is tot die set users en items.


## Step 3 - make_graph_object

Met deze functie parsen we tot slot die subset van de dataframe naar het dataformaat dat we willen. Hier wordt set gemaakt van users, items en ook de oui en oiu (hoeveelste item van user het is). TODO: Hier moet ook een opsplitsing worden gemaakt voor elke t, oftwel na elk item dat de target user heeft gekocht. Op dit moment is het alleen voor t=-1.

Zorgen/mysteries om nog op te lossen

- in het paper halen ze users met > x transacties weg, dat is nu nog lastig omdat het voor de lookup table een continuus range moet zijn

- users kunnen meer dan n transacties hebben in de gesamplede ding

- hoort het per timestep op nieuw gesampled te worden? wij samplen een keer voor einde van dataset en snijden dan het weg

In [4]:
# create dictionaries

# n is max number of recent transactions per node sampled
n = 10

# -- build User most recent transaction lists --
u_connection_list = [np.zeros(n)]
u_transaction_list = [np.zeros(n)]

for u, us_transactions in df.groupby('u'):
    bought = us_transactions['i'].values[-n:]
    
    zero_padded = np.zeros(n)
    zero_padded[:len(bought)] = bought + 1 # offset by 1 for dummy
    
    u_connection_list.append(zero_padded)
    
    transaction_idx = us_transactions['i'].index.values[-n:]
    
    zero_padded_t = np.zeros(n)
    zero_padded_t[:len(transaction_idx)] = transaction_idx
    
    u_transaction_list.append(zero_padded_t)

print("Created user dictionaries")

# -- build Item most recent transaction lists --
i_connection_list = [np.zeros(n)]
i_transaction_list = [np.zeros(n)]

for i, is_transactions in df.groupby('i'):
    bought = is_transactions['u'].values[-n:]
    
    zero_padded = np.zeros(n)
    zero_padded[:len(bought)] = bought + 1 # offset by 1 for dummy
    
    i_connection_list.append(zero_padded)
    
    transaction_idx = is_transactions['u'].index.values[-n:]
    
    zero_padded_t = np.zeros(n)
    zero_padded_t[:len(transaction_idx)] = transaction_idx
    
    i_transaction_list.append(zero_padded_t)

print("Created item dictionaries")

# -- parse to array --

u_connections = np.stack(u_connection_list).astype(np.int32)
u_transactions = np.stack(u_transaction_list).astype(np.int32)

i_connections = np.stack(i_connection_list).astype(np.int32)
i_transactions = np.stack(i_transaction_list).astype(np.int32)

print(f"Created database of {len(u_connections)-1} users and {len(i_connections)-1} items")

Created user dictionaries
Created item dictionaries
Created database of 52204 users and 57289 items


In [5]:
all_users = df['u'].unique()
f"There are {len(all_users)} users"

'There are 52204 users'

In [6]:
# get user network

def get_user_network(index, m=2):
    # u_m and i_m are the sets of explored nodes
    u_m = np.array([0]) # 0 is dummy
    i_m = np.array([0])
    
    # transactions of sampled nodes
    transactions_m = np.array([0])
    
    # u_temp and i_temp are the sets of unexplored nodes
    u_temp = np.array([index+1]) # initialize as the given index
    i_temp = u_connections[u_temp] # initialize as its purchases
    
    # add initialized purchases to transaction base
    new_transactions = u_transactions[u_temp].flatten()
    transactions_m = np.union1d(transactions_m, new_transactions)
        
    for j in range(m):
        new_users = np.unique(i_connections[i_temp])
        u_temp = np.union1d(u_temp, new_users)
        
        new_transactions = i_transactions[i_temp].flatten()
        transactions_m = np.union1d(transactions_m, new_transactions)
        
        u_temp = np.setdiff1d(u_temp, u_m, assume_unique=True)
        u_m = np.union1d(u_m, u_temp)
        
        if len(u_temp)==0:
            break
            
        new_items = np.unique(u_connections[u_temp])
        i_temp = np.union1d(i_temp, new_items)
        
        new_transactions = u_transactions[u_temp].flatten()
        transactions_m = np.union1d(transactions_m, new_transactions)
        
        i_temp = np.setdiff1d(i_temp, i_m, assume_unique=True)
        i_m = np.union1d(i_temp, i_m)
        
        if len(i_temp)==0:
            break
    
    # [1:] to ignore first element since its dummy 0
    # -1 to offset back (it was offset to allow for dummy 0)
    return u_m[1:]-1, i_m[1:]-1, transactions_m[1:]

user_ids, item_ids, transaction_ids = get_user_network(41)

display("user_ids:", user_ids, user_ids.shape, "item_ids:", item_ids, item_ids.shape, "trans_ids:", transaction_ids, transaction_ids.shape)

'user_ids:'

array([   39,    40,    41, ..., 52093, 52094, 52095])

(2077,)

'item_ids:'

array([   17,    43,    84, ..., 57264, 57267, 57277])

(7941,)

'trans_ids:'

array([   339,    340,    341, ..., 394238, 394239, 394240])

(14131,)

In [7]:
# # benchmark run on all users
# st = time.time()
# selected_nodes = {}
# for u in tqdm(all_users[::-1]):
#     users, items, trans = get_user_network(u)
    
#     selected_nodes[u] = trans
    
# print(f"{time.time()-st} seconds")

In [8]:
# step 3 make graph object

In [9]:
def compute_oui(df):
    """
    oui = o_u^i = order of u−i interaction
    = the position of item i in all items that the u has interacted with
    
    i is u's item #oui
    """
    return df.groupby("u")["t"].rank("first")

def compute_oiu(df):
    """
    oiu = o_i^u = order of i−u interaction
    = the position of user u in all users that the i has interacted with
    
    u is i's buyer #oiu
    """
    return df.groupby("i")["t"].rank("first")

In [10]:
def make_graph_object(user_index, transaction_ids):
    """
    Makes PyTorch Heterograph not temporal for now
    """
    data = HeteroData()
    
    sub_df = df.loc[transaction_ids]

    # get important transactions
    user_sequence_df = sub_df[sub_df['u']==user_index]
    final_item = user_sequence_df.iloc[-1]

    # remove transactions from the future
    sub_df = sub_df[sub_df['t'] < final_item['t']]
    
    if len(sub_df) < 1:
        return data

    # make graph
    
    # remap
    mapping_u = {u_id : i for i, u_id in enumerate(sub_df['u'].unique())}
    mapping_i = {i_id : i for i, i_id in enumerate(sub_df['i'].unique())}

    sub_df['u'] = sub_df['u'].map(mapping_u)
    sub_df['i'] = sub_df['i'].map(mapping_i)
    
    # sort dataframe in same order as pytorch geometric sorts (!!)
    sub_df = sub_df.sort_values(['u', 'i'])

    # make edge index
    users = torch.tensor(sub_df['u'].values)
    items = torch.tensor(sub_df['i'].values)

    # make edge weights
    relative_time = final_item['t'] - sub_df['t'].values
    weights = torch.tensor(1 - relative_time / max(relative_time))**3 # SHOULD BE EXPERIMENTED WITH this is unofficial
    
    # build object
    
    data['u'].x = torch.tensor(list(mapping_u.keys()))
    data['i'].x = torch.tensor(list(mapping_i.keys()))

    data['u', 'bought', 'i'].edge_index = torch.sparse_coo_tensor(
        torch.stack((users, items)),
        weights,
        size=(len(mapping_u), len(mapping_i))
    ).coalesce()
    
    data['u', 'bought', 'i'].oui = torch.tensor(compute_oui(sub_df).values, dtype=int) # can maybe be done beforehand TODO
    data['u', 'bought', 'i'].oiu = torch.tensor(compute_oiu(sub_df).values, dtype=int)
    
    data.y = final_item['i']   
    
    return data

user_ids, item_ids, transaction_ids = get_user_network(41)
make_graph_object(41, transaction_ids)

HeteroData(
  y=17,
  u={ x=[2030] },
  i={ x=[7444] },
  (u, bought, i)={
    edge_index=[2030, 7444],
    oui=[13074],
    oiu=[13074]
  }
)

In [11]:
# benchmark

np.random.shuffle(all_users)
min_graph_size = 100


st = time.time()

graphs = []

failed = 0
for u in tqdm(all_users[:250]):
    user_ids, item_ids, transaction_ids = get_user_network(u)
    
    if len(transaction_ids) < min_graph_size:
        failed += 1
        continue
        
    graph = make_graph_object(u, transaction_ids)
    
    del(user_ids)
    del(item_ids)
    del(transaction_ids)
    
    if len(graph) > 0:
        graphs.append(graph)
    
print(f"{time.time()-st} seconds")

  0%|          | 0/250 [00:00<?, ?it/s]

4.147935152053833 seconds


# DGSR

In [12]:
import torch.nn as nn
import torch.nn.functional as F

In [13]:
user_ids, item_ids, transaction_ids = get_user_network(41)
graph = make_graph_object(41, transaction_ids)

In [152]:
def sparse_dense_mul(s, d):
    """
    elementwise multiply sparse and dense matrix of same size
    
    Parameters:
        s: sparse matrix
        d: dense matrix
    """
    i = s._indices()
    v = s._values()
    dv = d[i[0,:], i[1,:]]  # get values from relevant entries of dense matrix
    return torch.sparse_coo_tensor(i, v * dv, s.size())


def pass_messages_no_possitional(messages, adjacency):
    """
    add messages together based on adjacency matrix
    
    Parameters:
        messages: tensor (i, h)
        adjacency: sparse tensor (u, i)
    """
    # parse adjacency matrix
    user_per_trans, item_per_trans = adjacency._indices()
    alpha = adjacency._values().unsqueeze(-1)
    
    # prepare output
    output = torch.zeros((adjacency.shape[0], messages.shape[1]), dtype=float)
    
    # add messages
    output.index_add_(0, user_per_trans, messages[item_per_trans] * alpha)
    
    # add embeddings
    output.index_add_(0, user_per_trans, alpha)
    
    # TODO: also scale by edge weight ?
    
    return output

def pass_messages(messages, adjacency, pVui):
    """
    add messages together based on adjacency matrix
    
    Parameters:
        messages: tensor (i, h)
        adjacency: sparse tensor (u, i)
        pVui: tensor (t, h)
    """
    # parse adjacency matrix
    user_per_trans, item_per_trans = adjacency._indices()
    alpha = adjacency._values().unsqueeze(-1)
    
    # prepare output
    output = torch.zeros((adjacency.shape[0], messages.shape[1]), dtype=float)
    
    # add messages
    output.index_add_(0, user_per_trans, messages[item_per_trans] * alpha)
    
    # add embeddings
    output.index_add_(0, user_per_trans, pVui * alpha)
    
    # TODO: also scale by edge weight ?
    
    return output

def relative_order(oui, by_who):
    """
    Parameters:
        oui (or oiu) : tensor (t) (the order #'s of each transaction)
        by_who : tensor (t) (the one to count the # of neighbours from)
    """    
    # compute amount of transactions of each user
    neighbourhood_sizes = torch.bincount(by_who)
    neighbourhood_sizes_per_trans = neighbourhood_sizes[by_who]

    # relative_order = Neighbourhood_size - order (zodat nieuwste altijd hetzelfde hebben)
    rui = torch.clip(neighbourhood_sizes_per_trans, max=n) - torch.clip(oui, max=n)
    
    return rui

def get_last(by_who, what, code):
    
    # compute amount of transactions of each user
    neighbourhood_sizes = torch.bincount(by_who)
    
    # compute cumulative indices of user
    cum_ind = torch.cumsum(neighbourhood_sizes, dim=0) - 1
    
    # select indices out of preferred transactions
    last_indices = torch.index_select(what, 0, cum_ind)
    
    # get item id's from graph
    last_ids = torch.index_select(code, 0, last_indices)
    
    return last_ids
    
       
class DGRNLayer(nn.Module): # Dynamic Graph Recommendation Network
    def __init__(self,
                 user_num, item_num,
                 hidden_size,
                 user_max, item_max
                ):
        super().__init__()
        """ init """
        self.user_vocab_num = user_num
        self.item_vocab_num = item_num
        
        self.user_max = user_max
        self.item_max = item_max
        
        self.hidden_size = hidden_size
        self.sqrt_d = np.sqrt(self.hidden_size)
        
        """ layers """        
        self.w1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False) # Long Term User
        self.w2 = nn.Linear(self.hidden_size, self.hidden_size, bias=False) # Long Term Item
        
        self.w3 = nn.Linear(self.hidden_size, self.hidden_size, bias=False) # Short Term User
        self.w4 = nn.Linear(self.hidden_size, self.hidden_size, bias=False) # Short Term Item
        
        self.pV = nn.Embedding(self.user_max, self.hidden_size) # user positional embedding
        self.pK = nn.Embedding(self.item_max, self.hidden_size) # item positional embedding
        
        self.last_user_embedding = nn.Embedding(self.user_vocab_num, self.hidden_size)
        self.last_item_embedding = nn.Embedding(self.item_vocab_num, self.hidden_size)
        
    def longterm(self, u_embedded, i_embedded, edge_index, rui, riu):
        # --- long term ---
        
        user_messages = self.w2(u_embedded) # (u, h)
        item_messages = self.w1(i_embedded) # (i, h)
        
        # message similarity
        e = (user_messages) @ (item_messages).T # (u, i)
        e = sparse_dense_mul(edge_index, e) # (u, i)
        
        user_per_trans, item_per_trans = edge_index.indices()
        
        # - users to items -
            
        # compute positional embeddings
        pVui = self.pV(rui)
        
        # dot product van elke pos embedding met betreffende user
        u_at_pVui = torch.einsum('ij, ij->i', user_messages[user_per_trans], pVui)
        
        # alpha is softmax(wu @ wi.T + wu @ p)
        e_ui = torch.sparse_coo_tensor(e._indices(), e._values() + u_at_pVui, e.size())  
        alphas = torch.sparse.softmax(e_ui / self.sqrt_d, dim=1) # (u, i)
        
        
        # - items to users -
        
        # compute positional embeddings
        pKiu = self.pK(riu)
        
        # dot product van elke pos embedding met betreffende user
        u_at_pKiu = torch.einsum('ij, ij->i', item_messages[item_per_trans], pKiu)
        
        # beta is softmax(wi @ wu.T + wi @ p)
        e_trans = torch.transpose(e, 0, 1)
        e_iu = torch.sparse_coo_tensor(e_trans._indices(), e_trans._values() + u_at_pKiu, e_trans.size())        
        betas = torch.sparse.softmax(e_iu / self.sqrt_d, dim=1) # (u, i)
        
        # pass messages
        longterm_hu = pass_messages(item_messages, alphas, pKiu)
        longterm_hi = pass_messages(user_messages, betas, pVui)
        
        return longterm_hu, longterm_hi
    
    def shortterm(self, u_embedded, i_embedded, edge_index, oui, oiu):
        """ TODO """
        
        # --- short term ---
        user_per_trans, item_per_trans = edge_index.indices()
        
        user_messages = self.w2(u_embedded) # (u, h)
        item_messages = self.w1(i_embedded) # (i, h)
        
        # Get last item
        last_item = get_last(user_per_trans, item_per_trans, graph['i'].x)
        last_item_embedding = self.last_item_embedding(last_item)
        last_item = self.w3(last_item_embedding)
        
        # Get last user from items
        last_user = get_last(item_per_trans, user_per_trans, graph['u'].x)
        last_user_embedding = self.last_user_embedding(last_user)
        last_user = self.w4(last_user_embedding)
        
        # message similarity alpha
        a = (last_item) @ (item_messages).T # (u, i)
        a = sparse_dense_mul(edge_index, a) # (u, i)
        
        # message similarity beta
        b = (last_user) @ (item_messages).T # (u, i)
        b = sparse_dense_mul(edge_index, b) # (u, i)
        
        
        # compute alphas
        a = torch.sparse_coo_tensor(a._indices(), a._values(), a.size())        
        alphas = torch.sparse.softmax(a / self.sqrt_d, dim=1) # (u, i)
        
        
        # compute betas
        b_trans = torch.transpose(b, 0, 1)
        b = torch.sparse_coo_tensor(b_trans._indices(), b_trans._values(), b_trans.size())  
        betas = torch.sparse.softmax(b / self.sqrt_d, dim=1) # (u, i)

        
        # pass messages
        shortterm_hu = pass_messages_no_possitional(item_messages, alphas)
        shortterm_hi = pass_messages_no_possitional(user_messages, betas)

        
        return shortterm_hu, shortterm_hi
        
        
    def forward(self, u_emb, i_emb, edge_index, rui, riu, oui, oiu):
        # propagate information
        # longterm
        hLu, hLi = self.longterm(u_emb, i_emb, edge_index, rui, riu)
        
        # shortterm
        hSu, hSi = self.shortterm(u_emb, i_emb, edge_index, oui, oiu)
        
        return hLu, hSu, hLi, hSi
        
class DGSRNetwork(nn.Module): # Dynamic Graph Recommendation Network
    def __init__(self,
                 user_num, item_num,
                 hidden_size,
                 user_max, item_max,
                 num_DGRN_layers
                ):
        super().__init__()
        """ init """
        self.user_vocab_num = user_num
        self.item_vocab_num = item_num
        
        self.user_max = user_max
        self.item_max = item_max
        
        self.hidden_size = hidden_size
        self.sqrt_d = np.sqrt(self.hidden_size)
        
        """ layers """
        # embedding
        self.user_embedding = nn.Embedding(self.user_vocab_num, self.hidden_size)
        self.item_embedding = nn.Embedding(self.item_vocab_num, self.hidden_size)
        
        # propogation
        self.DGSRLayers = nn.ModuleList()
        for _ in range(num_DGRN_layers):
            self.DGSRLayers.append(DGRNLayer(user_num, item_num, hidden_size, user_max, item_max))
        
        # node updating
        self.w3 = nn.Linear(self.hidden_size*3, self.hidden_size, bias=False)
        self.w4 = nn.Linear(self.hidden_size*3, self.hidden_size, bias=False)
        
        # recommendation
        self.wP = nn.Linear(self.hidden_size, self.hidden_size*(num_DGRN_layers+1), bias=False)
        
    def forward(self, graph):
        
        # embedding
        hu = self.user_embedding(graph['u'].x) # (u, h)
        hi = self.item_embedding(graph['i'].x) # (i, h)
        
        # parse graph
        edges = graph['u', 'bought', 'i'].edge_index
        oui = graph['u', 'bought', 'i'].oui
        oiu = graph['u', 'bought', 'i'].oiu
        
        user_per_trans, item_per_trans = edges.indices()
        
        rui = relative_order(oui, user_per_trans)
        riu = relative_order(oiu, item_per_trans)
        
        # propogation
        
        # iterate over Dynamic Graph Sequential Recommendation Layers
        hu_list = [hu]
        hi_list = [hi]
        for DGSR in self.DGSRLayers:
            hLu, hSu, hLi, hSi = DGSR(hu, hi, edges, rui, riu, oui, oiu)
            
            # concatenate information
            hu_concat = torch.hstack((hLu, hSu, hu)).float()
            hi_concat = torch.hstack((hLi, hSi, hi)).float()
            
            # make new embedding
            hu = torch.tanh(self.w3(hu_concat))
            hi = torch.tanh(self.w4(hi_concat))
            
            # save user embedding at every timestep
            hu_list.append(hu)
            hi_list.append(hi)
        
        # recommendation
        prediction_user_embedding = torch.hstack(hu_list)
        
        scores = prediction_user_embedding @ self.wP(hi_list[0]).T
        predictions = torch.softmax(scores, 0)
        
        return predictions
        
          
"""
Make network
"""
user_num = len(df['u'].unique())
item_num = len(df['i'].unique())

hidden_size = 64


network = DGSRNetwork(user_num, item_num, hidden_size, user_max=n, item_max=n, num_DGRN_layers=1)

"""
Forward that shit
"""
graph = graphs[0]
out = network(graph)

out, out.shape

(tensor([[9.9685e-06, 1.0990e-10, 6.9780e-07,  ..., 3.6283e-07, 9.1665e-05,
          6.0042e-10],
         [3.2736e-06, 2.7381e-07, 3.1657e-08,  ..., 4.3127e-04, 3.7399e-09,
          1.7151e-06],
         [5.2343e-05, 2.8206e-07, 1.6428e-09,  ..., 1.2057e-06, 3.1164e-04,
          4.6403e-10],
         ...,
         [1.6133e-08, 1.5432e-07, 7.1708e-11,  ..., 6.7970e-04, 2.8008e-05,
          2.0587e-11],
         [8.1203e-13, 1.8312e-09, 3.0792e-12,  ..., 3.6410e-07, 5.5587e-12,
          2.4694e-08],
         [1.7497e-12, 7.9909e-05, 3.0776e-06,  ..., 2.4634e-04, 3.8738e-11,
          5.2122e-05]], grad_fn=<SoftmaxBackward0>),
 torch.Size([632, 1656]))